In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm import tqdm

from Blocks import Transformer, Data, MeanNormalization, get_size

In [11]:
data = torch.load("Data.pt")

temp = list(data.columns)
cols = temp[1:]
cols.append("Label")
data = data[cols].drop(columns = ["Close", "Adj Close"])
data = data.astype(float)
data.head(2)

,Pos1,Pos2,Pos3,Pos4,Pos5,Pos6,Pos7,Pos8,Pos9,Pos10,...,Neu21,Neu22,Neu23,Neu24,Neu25,Open,High,Low,Volume,Label
Date,,,,,,,,,,,,,,,,,,,,,
2008-08-08,0.066467,0.020223,0.043872,0.021866,0.01509,0.023724,0.051096,0.026434,0.012361,0.040297,...,0.873544,0.204416,0.538325,0.487061,0.411749,11432.089844,11759.959961,11388.040039,212830000.0,0.0
2008-08-11,0.071691,0.049347,0.932647,0.034207,0.02592,0.025570,0.018784,0.017460,0.055654,0.025160,...,0.534569,0.183160,0.895037,0.493829,0.930178,11729.669922,11867.110352,11675.530273,183190000.0,1.0


In [12]:
data = torch.FloatTensor(data.values)
data.shape

torch.Size([1989, 80])

In [4]:
n = 0.85    #Train data precentage
rows = data.shape[0]

train_data = data[:int(rows*n),:]
test_data = data[int(rows*n):,:]
train_data.shape, test_data.shape

(torch.Size([1690, 82]), torch.Size([299, 82]))

In [5]:
d_set_train = Data(train_data, win_len=30, stride=1, transform = MeanNormalization())
d_set_test = Data(test_data, win_len=10, stride=1, transform = MeanNormalization())

torch.Size([1690, 82])
torch.Size([299, 82])


In [6]:
trainloader = DataLoader(dataset=d_set_train, batch_size = 16)
testloader = DataLoader(dataset=d_set_test, batch_size = 16)

In [7]:
device = torch.device('cuda:0')
model = Transformer(embed_dim=81, enc_expansion=4, dec_expansion=4, enc_heads=8, dec_heads=8,
                    num_enc=6, num_dec=6, pos_enc=True, device=device)

In [8]:
get_size(model)

'model size: 40626368 / bit | 5.08 / MB'

In [10]:
torch.std(d_set_train[0][0][:,5])

tensor(1.)

<center><h1>Training

In [9]:
from torch import optim

In [10]:
model.to(device)
criterion = nn.BCELoss()

optimizer = optim.AdamW(model.parameters(), lr = 0.1)
epochs = 10

In [11]:
for _ in tqdm(range(epochs)):
    for x,y in trainloader:
        x = x.to(device)
        y = y.to(device)

        yhat = model(x, y)
        loss = criterion(yhat, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [01:03<09:34, 63.79s/it]


KeyboardInterrupt: 